# LangChain LangSmith and LangChain Hub

-   Find the  [Notebook](https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2008%20-%20LangSmith_Introduction.ipynb)  for this section at  [towardsai.net/book](http://towardsai.net/book).

💡While LangChain is appropriate for initial prototyping, LangSmith provides a setting for debugging, testing, and refining LLM applications.

[LangSmith](https://www.langchain.com/langsmith)  is a platform for evaluating and monitoring the quality of LLM systems’ outputs. Its functionality includes tracking metadata, token usage, and execution time, which is vital for managing resources effectively.

LangSmith improves the efficiency and performance of new chains and tools. It also provides visualization tools to recognize response patterns and trends, enhancing the understanding and analysis of performance. It allows users to create customized testing environments tailored to specific requirements, enabling comprehensive evaluation under various conditions. The platform also tracks the executions linked to an active instance and allows for the testing and assessing of any prompts or responses produced. LangSmith offers several tutorials and in-depth documentation to assist users in getting started.

On the other hand, LangChain Hub is a platform designed for the development, storage, and sharing of reusable components used in building applications with large language models (LLMs). It offers a repository of modules such as prompts, chains, and agents that can be easily accessed and integrated into projects. LangChain Hub simplifies the creation of LLM-driven applications by providing tools and templates and promoting collaboration among developers.

This section contains an example of how to use LangChain Hub together with a question-answering chain in LangChain. We will guide you through the setup process for LangChain, including installing necessary libraries and configuring environment variables. A LangSmith account is required for certain features like tracing. Detailed steps for setting up a new account will be provided.

First, you will need an API key. Navigate to the  [LangSmith](https://www.langchain.com/langsmith)  website and register for an account. Find the option to generate an API key on the settings page. Click the “Generate API Key” button to receive your API key.

Install the necessary libraries using the command `!pip install -q langchain==0.0.346 openai==1.3.7 tiktoken==0.5.2 cohere==4.37 deeplake==3.8.11 langchainhub==0.1.14`.

Configure the environment with the API keys for OpenAI, which will be used in the embedding generation process, and the Activeloop key, which will be used to store data in the cloud:

In [ ]:
from advanced_rag_custom_utils.helper import (
    get_openai_api_key, 
    get_activeloop_api_key, 
    get_langchain_api_key,
    get_langchain_project,
    get_langchain_tracing_v2,
    get_langchain_endpoint,
)

OPENAI_API_KEY = get_openai_api_key()
ACTIVELOOP_API_KEY = get_activeloop_api_key()

LANGCHAIN_API_KEY = get_langchain_api_key()
LANGCHAIN_TRACING_V2 = get_langchain_tracing_v2()
LANGCHAIN_PROJECT = get_langchain_project()
LANGCHAIN_ENDPOINT = get_langchain_endpoint()

The following code shows how to commit a prompt to the LangChain Hub by adding it to your handle’s namespace.

In [ ]:
from langchain import hub
from langchain.prompts.chat import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:""")

handle = "<YOUR_USERNAME>"
hub.push(f"{handle}/rag-prompt", prompt)

If you update the prompt, you can push the modified prompt to the same key to “commit” a new version of the prompt during evaluation. Let’s say we want to add a system message to the prompt.

In [ ]:
# You may try making other changes and saving them in a new commit.
from langchain import schema

prompt.messages.insert(0,
   schema.SystemMessage(
       content="You are a precise, autoregressive question-answering system."
   )
  )

The most recent version of the prompt is kept as the latest version.

In [ ]:
# Pushing to the same prompt "repo" will create a new commit
hub.push(f"{handle}/rag-prompt", prompt)

LangSmith also offers a feature for reviewing the inputs and outputs of each component within a chain, making it easier to log runs for large language model applications. This functionality is particularly beneficial when debugging your application or understanding the behavior of certain components. For more information on  [Tracing](http://towardsai.net/book), visit the LangChain documentation.

The next steps involve loading data from a webpage, dividing it into smaller chunks, converting these segments into embeddings, and then storing them in the Deep Lake vector database. This process also includes prompt templates from the  [LangSmith Hub](https://smith.langchain.com/hub).

The content of a webpage can be read using the `WebBaseLoader` class. This class will provide a single instance of the `Document` class with all the text from the URL. Subsequently, this large text is divided into smaller chunks, each comprising 500 characters without overlapping, resulting in 130 chunks.

In [ ]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Loading
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()
print(len(data))

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)
print(len(all_splits))

    1  
    130

Chunks can be saved in the Deep Lake vector database using the LangChain integration. The `DeepLake` class transforms texts into embeddings using `OpenAI’s` API and stores these results in the cloud. You can use your own organization name (your username by default) to create the dataset. Note that this task involves the costs of utilizing OpenAI endpoints.

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DeepLake

vectorstore = DeepLake.from_documents(
                        all_splits,
                        dataset_path="hub://genai360/langsmith_intro",
                        embedding=OpenAIEmbeddings(), overwrite=False)

    Your Deep Lake dataset has been successfully created!  
    Creating 130 embeddings in 1 batches of size 130:: 100%|██████████| 1/1 [00:05<00:00, 5.81s/it] dataset (path='hub://genai360/langsmith_intro', tensors=['text', 'metadata', 'embedding', 'id'])  
      
    tensor htype shape dtype compression  
    ------- ------- ------- ------- -------  
    text text (130, 1) str None  
    metadata json (130, 1) str None  
    embedding embedding (130, 1536) float32 None  
    id text (130, 1) str None

After processing the data, select a prompt from the LangChain Hub, getting a `ChatPromptTemplate` instance. Using a Prompt Template removes trial and error and allows using already-tested implementations. The code below tags a specific version of the prompt to ensure that future changes do not affect the version currently deployed.

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt:50442af1")
print(prompt)

    ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])


Finally, use the `RetrievalQA` chain to retrieve related documents from the database and then the `ChatOpenAI` model to build the final response using these documents.

In [ ]:
# LLM
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

question = "What are the approaches to Task Decomposition?"
result = qa_chain({"query": question})
result["result"]

    The approaches to task decomposition include using LLM with simple prompting, task-specific instructions, and human inputs.

Prompt versioning encourages continual experimentation and collaboration, preventing the unintentional deployment of chain components without thorough testing. Leveraging the LangChain Hub and prompt versioning provides an easy and organized approach to many LLM-related tasks. It ensures reliable performance by using pre-tested, versioned prompts, reducing the need for trial and error.